In [1]:
from dependencies import *

userdata = pd.read_csv('data/yelp/user.csv', low_memory=False)
uid2num = {k:v for k,v in zip(userdata['user_id'].values,range(len(userdata['user_id'].values)))}
num2uid = {k:v for v,k in uid2num.items()}

udata = pickle.load(open( "udata.p", "rb" ))
adj = {uid2num[k]:[uid2num[x] for x in v.split(', ') if x in uid2num] for k,v in zip(udata['user_id'].values,udata['friends'].values)}


Using TensorFlow backend.


In [2]:
components = pickle.load(open( "graph_components_apr29.p", "rb" ))
#Get the training and the test connected component
gc = components[0]

def giveagraph(gc,start=0,end=10000):
	temp = nx.Graph()
	temp.add_nodes_from(list(gc.nodes)[start:end])
	temp.add_edges_from([(i,j) for i in temp.nodes for j in gc[i] if j in temp.nodes ])
	return max(nx.connected_component_subgraphs(temp), key=len)

#traing = giveagraph(gc)
#testg = giveagraph(gc,10000,20000)



In [11]:
def giveagraph_nodes(gc,nodes):
	temp = nx.Graph()
	temp.add_nodes_from(nodes)
	temp.add_edges_from([(i,j) for i in nodes for j in gc[i] if j in nodes ])
	return max(nx.connected_component_subgraphs(temp), key=len)

import random
nodes  = list(gc.nodes)[0:20000]
random.Random(4).shuffle(nodes)
traing = giveagraph_nodes(gc,nodes[:10000])
testg = giveagraph_nodes(gc,nodes[10000:])


In [16]:
len(testg.nodes)

7151

In [17]:
def sample_edges(g,possize=100,negsize=100,full = False):
    negexamples = list(nx.non_edges(g))
    totaledges = len(g.edges())
    if full:
        posset = np.array(g.edges())
        negsize = totaledges
    else:
        np.random.seed(4)
        posset = np.random.choice(np.array(g.edges()), possize)
    np.random.seed(4)
    negindices = np.random.choice(range(len(negexamples)), negsize)
    negset = np.array(negexamples)[negindices]
    return posset,negset

postrainset, negtrainset = sample_edges(traing, full = True)
postestset, negtestset = sample_edges(testg, full = True)

In [20]:
len(postestset)

71839

In [21]:
ids  = udata.to_numpy()[:,[0]]
features = udata.to_numpy()[:,[2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21]]
uid2feat = dict(zip(ids.reshape(-1),features))

In [22]:
uid2fset = {a:set(b.split(', ')) for a,b in zip(udata['user_id'].values,udata['friends'].values)}
uid2bidset = {a:b for a,b in zip(udata['user_id'].values,udata['bidset'].values)}
uid2cityset = {a:b for a,b in zip(udata['user_id'].values,udata['cityset'].values)}

def givenumcommon(uid2set,uid1,uid2):
    a = uid2set[uid1]
    b = uid2set[uid2]
    return len(a.intersection(b))

def givenumunion(uid2set,uid1,uid2):
    a = uid2set[uid1]
    b = uid2set[uid2]
    return len(a.union(b))

In [23]:
def givejaccard(uidset,uid1,uid2):
    a = givenumcommon(uidset,uid1,uid2)
    b = givenumunion(uidset,uid1,uid2)
    return np.round((a*1.0)/(b*1.0),2)

def giveprefattach(uid2set,uid1,uid2):
    a = uid2set[uid1]
    b = uid2set[uid2]
    return (len(a) * len(b))

In [24]:
def giveedgefeats(pairlist,cfriends=True,cbids=True,ccities= True):
    X = []
    for p in pairlist:
        id0 = num2uid[p[0]]
        id1 = num2uid[p[1]]
        feats = []
        if cfriends:
            feats.append(givenumcommon(uid2fset,id0,id1))
        if cbids:
            feats.append(givenumcommon(uid2bidset,id0,id1))
        if ccities:
            feats.append(givenumcommon(uid2cityset,id0,id1))
        X.append(feats)
    return np.array(X)

def giveedgefeats_jaccard(pairlist):
    X = []
    for p in pairlist:
        id0 = num2uid[p[0]]
        id1 = num2uid[p[1]]
        feats = []
        feats.append(givejaccard(uid2fset,id0,id1))
        feats.append(givejaccard(uid2bidset,id0,id1))
        feats.append(givejaccard(uid2cityset,id0,id1))
        X.append(feats)
    return np.array(X)

def giveedgefeats_prefattach(pairlist):
    X = []
    for p in pairlist:
        id0 = num2uid[p[0]]
        id1 = num2uid[p[1]]
        feats = []
        feats.append(giveprefattach(uid2fset,id0,id1))
        feats.append(giveprefattach(uid2bidset,id0,id1))
        feats.append(giveprefattach(uid2cityset,id0,id1))
        X.append(feats)
    return np.array(X)

def giveedgefeats_all(pairlist):
    X = []
    for p in pairlist:
        id0 = num2uid[p[0]]
        id1 = num2uid[p[1]]
        feats = []
        feats.append(givenumcommon(uid2fset,id0,id1))
        feats.append(givenumcommon(uid2bidset,id0,id1))
        feats.append(givenumcommon(uid2cityset,id0,id1))
        feats.append(givejaccard(uid2fset,id0,id1))
        feats.append(givejaccard(uid2bidset,id0,id1))
        feats.append(givejaccard(uid2cityset,id0,id1))
        X.append(feats)
    return np.array(X)

def giveedgedata(posset,negset,cfriends=True,cbids=True,ccities= True):
    X1  = giveedgefeats(posset,cfriends,cbids,ccities)
    X2  = giveedgefeats(negset,cfriends,cbids,ccities)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y

def giveedgedata_jaccard(posset,negset):
    X1  = giveedgefeats_jaccard(posset)
    X2  = giveedgefeats_jaccard(negset)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y

def giveedgedata_prefattach(posset,negset):
    X1  = giveedgefeats_prefattach(posset)
    X2  = giveedgefeats_prefattach(negset)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y


def giveedgedata_all(posset,negset):
    X1  = giveedgefeats_all(posset)
    X2  = giveedgefeats_all(negset)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y

def givenodefeats(pairlist):
    X = []
    for p in pairlist:
        id0 = num2uid[p[0]]
        id1 = num2uid[p[1]]
        X.append(np.concatenate([uid2feat[id0], uid2feat[id1]]))
    return np.array(X)

def givenodedata(posset,negset):
    X1  = givenodefeats(posset)
    X2  = givenodefeats(negset)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y


def givedatausingbothfeats(posset,negset):
    X1  = np.concatenate((giveedgefeats(posset),givenodefeats(posset)),1)
    X2  = np.concatenate((giveedgefeats(negset),givenodefeats(negset)),1)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y


def givedatausingbothfeats_jaccard(posset,negset):
    X1  = np.concatenate((giveedgefeats_jaccard(posset),givenodefeats(posset)),1)
    X2  = np.concatenate((giveedgefeats_jaccard(negset),givenodefeats(negset)),1)
    X = np.concatenate((X1,X2),0)
    y = np.concatenate((np.ones((X1.shape[0],1)),np.zeros((X2.shape[0],1))),0)
    X,y = shuffle(X,y)
    return X,y


In [40]:
Xtrain_edge,ytrain_edge = giveedgedata(postrainset,negtrainset,cfriends=True,cbids=True,ccities= True)
Xtest_edge,Ytest_edge = giveedgedata(postestset,negtestset,cfriends=True,cbids=True,ccities= True)

In [83]:
Xtrain_edge_fc,ytrain_edge_fc = giveedgedata(postrainset,negtrainset,cfriends=True,cbids=False,ccities= True)
Xtest_edge_fc,Ytest_edge_fc = giveedgedata(postestset,negtestset,cfriends=True,cbids=False,ccities= True)
Xtrain_edge_fb,ytrain_edge_fb = giveedgedata(postrainset,negtrainset,cfriends=True,cbids=True,ccities= False)
Xtest_edge_fb,Ytest_edge_fb = giveedgedata(postestset,negtestset,cfriends=True,cbids=True,ccities= False)
Xtrain_edge_bc,ytrain_edge_bc = giveedgedata(postrainset,negtrainset,cfriends=False,cbids=True,ccities= True)
Xtest_edge_bc,Ytest_edge_bc = giveedgedata(postestset,negtestset,cfriends=False,cbids=True,ccities= True)

In [84]:
Xtrain_edge_f,ytrain_edge_f = giveedgedata(postrainset,negtrainset,cfriends=True,cbids=False,ccities= False)
Xtest_edge_f,Ytest_edge_f = giveedgedata(postestset,negtestset,cfriends=True,cbids=False,ccities= False)
Xtrain_edge_b,ytrain_edge_b = giveedgedata(postrainset,negtrainset,cfriends=False,cbids=True,ccities= False)
Xtest_edge_b,Ytest_edge_b = giveedgedata(postestset,negtestset,cfriends=False,cbids=True,ccities= False)
Xtrain_edge_c,ytrain_edge_c = giveedgedata(postrainset,negtrainset,cfriends=False,cbids=False,ccities= True)
Xtest_edge_c,Ytest_edge_c = giveedgedata(postestset,negtestset,cfriends=False,cbids=False,ccities= True)

In [82]:
Xtrain_edgeall,ytrain_edgeall = giveedgedata_all(postrainset,negtrainset)
Xtest_edgeall,Ytest_edgeall = giveedgedata_all(postestset,negtestset)

In [42]:
Xtrain_jaccard,ytrain_jaccard = giveedgedata_jaccard(postrainset,negtrainset)
Xtest_jaccard,Ytest_jaccard = giveedgedata_jaccard(postestset,negtestset)

In [56]:
Xtrain_both,ytrain_both = givedatausingbothfeats(postrainset,negtrainset)
Xtest_both,Ytest_both = givedatausingbothfeats(postestset,negtestset)

In [62]:
Xtrain_bothjaccard,ytrain_bothjaccard = givedatausingbothfeats_jaccard(postrainset,negtrainset)
Xtest_bothjaccard,Ytest_bothjaccard = givedatausingbothfeats_jaccard(postestset,negtestset)

In [67]:
Xtrain_node,ytrain_node = givenodedata(postrainset,negtrainset)
Xtest_node,Ytest_node = givenodedata(postestset,negtestset)

In [92]:
Xtrain_pref,ytrain_pref = giveedgedata_prefattach(postrainset,negtrainset)
Xtest_pref,Ytest_pref = giveedgedata_prefattach(postestset,negtestset)

In [45]:
def giveresults(Ytest,ytestpred):
    print("Accuracy ",np.round(accuracy_score(Ytest,ytestpred),3))
    print("Precision ",np.round(precision_score(Ytest,ytestpred),3))
    print("Recall ",np.round(recall_score(Ytest,ytestpred),3))
    print("F1 Score ",np.round(f1_score(Ytest,ytestpred),3))

In [75]:
def give_log_reg_results(Xtrain,ytrain,Xtest,Ytest):
    model = LogisticRegression(solver = 'lbfgs')
    model.fit(Xtrain,ytrain.reshape(-1,))
    ytestpred = model.predict(Xtest)
    giveresults(Ytest,ytestpred)

In [93]:
give_log_reg_results(Xtrain_pref,ytrain_pref,Xtest_pref,Ytest_pref)

Accuracy  0.5
Precision  0.5
Recall  1.0
F1 Score  0.667


In [86]:
give_log_reg_results(Xtrain_edge_fc,ytrain_edge_fc,Xtest_edge_fc,Ytest_edge_fc)

Accuracy  0.911
Precision  0.955
Recall  0.863
F1 Score  0.906


In [87]:
give_log_reg_results(Xtrain_edge_fb,ytrain_edge_fb,Xtest_edge_fb,Ytest_edge_fb)

Accuracy  0.909
Precision  0.962
Recall  0.851
F1 Score  0.903


In [88]:
give_log_reg_results(Xtrain_edge_bc,ytrain_edge_bc,Xtest_edge_bc,Ytest_edge_bc)

Accuracy  0.742
Precision  0.793
Recall  0.655
F1 Score  0.718


In [89]:
give_log_reg_results(Xtrain_edge_f,ytrain_edge_f,Xtest_edge_f,Ytest_edge_f)

Accuracy  0.904
Precision  0.964
Recall  0.839
F1 Score  0.897


In [90]:
give_log_reg_results(Xtrain_edge_b,ytrain_edge_b,Xtest_edge_b,Ytest_edge_b)

Accuracy  0.728
Precision  0.872
Recall  0.535
F1 Score  0.663


In [91]:
give_log_reg_results(Xtrain_edge_c,ytrain_edge_c,Xtest_edge_c,Ytest_edge_c)

Accuracy  0.685
Precision  0.796
Recall  0.497
F1 Score  0.612


In [79]:
give_log_reg_results(Xtrain_edge,ytrain_edge,Xtest_edge,Ytest_edge)

Accuracy  0.912
Precision  0.956
Recall  0.864
F1 Score  0.908


In [85]:
give_log_reg_results(Xtrain_edgeall,ytrain_edgeall,Xtest_edgeall,Ytest_edgeall)

Accuracy  0.913
Precision  0.957
Recall  0.866
F1 Score  0.909


In [80]:
give_log_reg_results(Xtrain_jaccard,ytrain_jaccard,Xtest_jaccard,Ytest_jaccard)

Accuracy  0.871
Precision  0.95
Recall  0.783
F1 Score  0.858


In [76]:
give_log_reg_results(Xtrain_node,ytrain_node,Xtest_node,Ytest_node)

Accuracy  0.735
Precision  0.699
Recall  0.827
F1 Score  0.757


/Users/shivankgoel/Documents/Anaconda/anaconda3/envs/nlpenv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [77]:
give_log_reg_results(Xtrain_both,ytrain_both,Xtest_both,Ytest_both)

Accuracy  0.875
Precision  0.866
Recall  0.888
F1 Score  0.877


/Users/shivankgoel/Documents/Anaconda/anaconda3/envs/nlpenv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [78]:
give_log_reg_results(Xtrain_bothjaccard,ytrain_bothjaccard,Xtest_bothjaccard,Ytest_bothjaccard)

Accuracy  0.736
Precision  0.698
Recall  0.831
F1 Score  0.759


/Users/shivankgoel/Documents/Anaconda/anaconda3/envs/nlpenv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [39]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [41]:
from keras.models import Sequential
from keras.layers import Dense

from numpy.random import seed
seed(3)
from tensorflow import set_random_seed
set_random_seed(3)

In [94]:
def giveedgemodel():
    model = Sequential()
    model.add(Dense(3, input_dim=3, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(2, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [97]:
def giveedgemodel_all():
    model = Sequential()
    model.add(Dense(4, input_dim=6, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(2, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [95]:
model = giveedgemodel()
model.fit(Xtrain_edge, ytrain_edge, epochs=10, batch_size=100,  verbose=2)
ytestpred = [round(x[0]) for x in model.predict(Xtest_edge)]
giveresults(Ytest_edge,ytestpred)

Epoch 1/10
 - 8s - loss: 0.3582 - acc: 0.9144
Epoch 2/10
 - 5s - loss: 0.2404 - acc: 0.9301
Epoch 3/10
 - 5s - loss: 0.2076 - acc: 0.9312
Epoch 4/10
 - 5s - loss: 0.1956 - acc: 0.9323
Epoch 5/10
 - 5s - loss: 0.1913 - acc: 0.9321
Epoch 6/10
 - 5s - loss: 0.1890 - acc: 0.9318
Epoch 7/10
 - 5s - loss: 0.1875 - acc: 0.9319
Epoch 8/10
 - 5s - loss: 0.1865 - acc: 0.9322
Epoch 9/10
 - 5s - loss: 0.1862 - acc: 0.9320
Epoch 10/10
 - 5s - loss: 0.1859 - acc: 0.9320
Accuracy  0.932
Precision  0.92
Recall  0.946
F1 Score  0.933


In [98]:
model = giveedgemodel_all()
model.fit(Xtrain_edgeall, ytrain_edgeall, epochs=10, batch_size=100,  verbose=2)
ytestpred = [round(x[0]) for x in model.predict(Xtest_edgeall)]
giveresults(Ytest_edgeall,ytestpred)

Epoch 1/10
 - 9s - loss: 0.3633 - acc: 0.9211
Epoch 2/10
 - 6s - loss: 0.2416 - acc: 0.9289
Epoch 3/10
 - 6s - loss: 0.2102 - acc: 0.9297
Epoch 4/10
 - 6s - loss: 0.1978 - acc: 0.9301
Epoch 5/10
 - 5s - loss: 0.1928 - acc: 0.9312
Epoch 6/10
 - 5s - loss: 0.1901 - acc: 0.9323
Epoch 7/10
 - 6s - loss: 0.1883 - acc: 0.9322
Epoch 8/10
 - 6s - loss: 0.1870 - acc: 0.9320
Epoch 9/10
 - 6s - loss: 0.1861 - acc: 0.9321
Epoch 10/10
 - 6s - loss: 0.1854 - acc: 0.9321
Accuracy  0.932
Precision  0.919
Recall  0.947
F1 Score  0.933


In [54]:
model = giveedgemodel()
model.fit(Xtrain_jaccard, ytrain_jaccard, epochs=10, batch_size=100,  verbose=2)

Epoch 1/10
 - 3s - loss: 0.6373 - acc: 0.6909
Epoch 2/10
 - 2s - loss: 0.4145 - acc: 0.8442
Epoch 3/10
 - 2s - loss: 0.3193 - acc: 0.8895
Epoch 4/10
 - 2s - loss: 0.2967 - acc: 0.9072
Epoch 5/10
 - 2s - loss: 0.2912 - acc: 0.9078
Epoch 6/10
 - 2s - loss: 0.2902 - acc: 0.9081
Epoch 7/10
 - 2s - loss: 0.2901 - acc: 0.9082
Epoch 8/10
 - 2s - loss: 0.2901 - acc: 0.9083
Epoch 9/10
 - 2s - loss: 0.2901 - acc: 0.9084
Epoch 10/10
 - 2s - loss: 0.2901 - acc: 0.9082


In [55]:
ytestpred = [round(x[0]) for x in model.predict(Xtest_jaccard)]
giveresults(Ytest_jaccard,ytestpred)

Accuracy  0.906
Precision  0.934
Recall  0.874
F1 Score  0.903


In [60]:
def givemodelusingbothfeats():
    model = Sequential()
    model.add(Dense(26, input_dim=39, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(18, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    
model = givemodelusingbothfeats()
model.fit(Xtrain_both, ytrain_both, epochs=10, batch_size=100,  verbose=2)

Epoch 1/10
 - 5s - loss: 0.3213 - acc: 0.8797
Epoch 2/10
 - 4s - loss: 0.2248 - acc: 0.9186
Epoch 3/10
 - 4s - loss: 0.2130 - acc: 0.9245
Epoch 4/10
 - 4s - loss: 0.2055 - acc: 0.9272
Epoch 5/10
 - 4s - loss: 0.2003 - acc: 0.9288
Epoch 6/10
 - 4s - loss: 0.2056 - acc: 0.9288
Epoch 7/10
 - 4s - loss: 0.1943 - acc: 0.9310
Epoch 8/10
 - 4s - loss: 0.1931 - acc: 0.9315
Epoch 9/10
 - 4s - loss: 0.1913 - acc: 0.9315
Epoch 10/10
 - 4s - loss: 0.1878 - acc: 0.9326


In [61]:
ytestpred = [round(x[0]) for x in model.predict(Xtest_both)]
giveresults(Ytest_both,ytestpred)

Accuracy  0.933
Precision  0.938
Recall  0.927
F1 Score  0.932


In [64]:
model = givemodelusingbothfeats()
model.fit(Xtrain_bothjaccard, ytrain_bothjaccard, epochs=10, batch_size=100,  verbose=2)

Epoch 1/10
 - 6s - loss: 0.4714 - acc: 0.8005
Epoch 2/10
 - 4s - loss: 0.4308 - acc: 0.8131
Epoch 3/10
 - 4s - loss: 0.4158 - acc: 0.8194
Epoch 4/10
 - 4s - loss: 0.4040 - acc: 0.8230
Epoch 5/10
 - 4s - loss: 0.3976 - acc: 0.8260
Epoch 6/10
 - 4s - loss: 0.3916 - acc: 0.8276
Epoch 7/10
 - 4s - loss: 0.3887 - acc: 0.8294
Epoch 8/10
 - 4s - loss: 0.3831 - acc: 0.8323
Epoch 9/10
 - 4s - loss: 0.3800 - acc: 0.8328
Epoch 10/10
 - 5s - loss: 0.3765 - acc: 0.8340


In [65]:
ytestpred = [round(x[0]) for x in model.predict(Xtest_bothjaccard)]
giveresults(Ytest_bothjaccard,ytestpred)

Accuracy  0.832
Precision  0.809
Recall  0.869
F1 Score  0.838


In [68]:
def givenodemodel():
    model = Sequential()
    model.add(Dense(24, input_dim=36, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(16, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    
model = givenodemodel()
model.fit(Xtrain_node, ytrain_node, epochs=10, batch_size=100,  verbose=2)

Epoch 1/10
 - 5s - loss: 0.4798 - acc: 0.7998
Epoch 2/10
 - 4s - loss: 0.4378 - acc: 0.8127
Epoch 3/10
 - 4s - loss: 0.4274 - acc: 0.8168
Epoch 4/10
 - 4s - loss: 0.4196 - acc: 0.8201
Epoch 5/10
 - 4s - loss: 0.4163 - acc: 0.8216
Epoch 6/10
 - 4s - loss: 0.4090 - acc: 0.8243
Epoch 7/10
 - 4s - loss: 0.4062 - acc: 0.8257
Epoch 8/10
 - 4s - loss: 0.4056 - acc: 0.8254
Epoch 9/10
 - 4s - loss: 0.4040 - acc: 0.8270
Epoch 10/10
 - 4s - loss: 0.4016 - acc: 0.8278


In [69]:
ytestpred = [round(x[0]) for x in model.predict(Xtest_node)]
giveresults(Ytest_node,ytestpred)

Accuracy  0.823
Precision  0.847
Recall  0.789
F1 Score  0.817
